# Pandas 에서 많이 사용하는 함수들

[Here are the 8 Functions I Use the Most](https://towardsdatascience.com/i-have-been-using-pandas-for-3-years-here-are-the-8-functions-i-use-the-most-4e54f4db56560) 참고

## 목차

1. [Pandas 로 csv 읽어오기](#1.-Pandas-로-csv-읽어오기)


2. [value_counts](#2.-value_counts) - 필드 별 개수 확인


3. [astype](#3.-astype) - 각 필드의 데이터 타입 확인


4. [isna & notna](#4.-isna-&-notna) - 결측치 확인


5. [dropna](#5.-dropna) - 결측치 제거


6. [fillna](#6.-fillna) - 결측치를 특정 값으로 처리


7. [groupby](#7.-groupby) - 특정 기준으로 그룹화하기


8. [unique & nunique](#8.-unique-&-nunique) - 고유값의 개수 확인하기


9. [정렬](#9.-정렬)

In [ ]:
import numpy as np
import pandas as pd

## 1. Pandas 로 csv 읽어오기

__기본__: pandas 에서 지원하는 데이터 파일을 읽어오는 함수

In [ ]:
df = pd.read_csv('data/test_data.CSV', encoding='cp949')
df.head()

__parse_dates__: 열 이름을 넣어주면 데이터 타입을 datatime64[ns]로 변환해준다.  
- `pd.to_datetime()` 함수로 datetime64 로 바꿀수도 있다.  
- '2019-01-01' 같은 `String` 형태의 자료형을 `datetime64` 로 바꿀 수 있다.

__usecols__: 여러 열 중 필요한 열만 가져온다.

In [ ]:
df = pd.read_csv('data/test_data.CSV', encoding='cp949', usecols=['이름', '나이'])
df.head()

__nrows__: 파일의 처음 n개의 행을 읽는다. 데이터 세트가 매우 큰 경우 유용하다.

In [ ]:
df = pd.read_csv('data/test_data.CSV', encoding='cp949', nrows=10)
df.shape

In [ ]:
df

##  2. value_counts

기본 함수에도 있던 함수. 범주형 열의 값 분포를 확인하는 데 유용하다.  
발생 횟수와 함께 열의 고유 값을 반환한다.

In [ ]:
df = pd.read_csv('data/test_data.CSV', encoding='cp949')
df["성별"].value_counts()

`normalize` 를 통해 분포를 비율로 확인할 수 있다.

In [ ]:
df["성별"].value_counts(normalize=True)

## 3. astype

데이터 분석 전 데이터 타입을 적절한 것으로 변환하는 건 거의 필수다.  
어떤 함수들은 적절한 데이터 타입에서 훨씬 더 효율적으로 동작하거나, 일부 함수들은 아예 특정 데이터 타입에서만 사용할 수 있다.

In [ ]:
df['이름'] = df['이름'].astype(str)
df.dtypes

string 형태는 object 로 출력된다.  

여러 열의 데이터 타입을 한 번에 변경하려면 딕셔너리를 사용하면 된다.

In [ ]:
df = df.astype({'성별': 'category', '직업': 'str'})
df.dtypes

## 4. isna & notna

[참고] - na 는 Numpy 의 Na(Not a Number)값을 의미함

데이터에 비어있는 값(결측값) 이 존재하는 경우가 있다.

`test_data_has_null.CSV` 파일을 열어서 확인해보자.

In [ ]:
df = pd.read_csv('data/test_data_has_null.CSV', encoding='cp949')
df.head()

위와 같이 결측값은 `NaN` 으로 출력된다.

`isna` 함수와 `notna` 함수는 DataFrame 내의 결측값을 확인해서 `bool` 형식으로 반환하는 함수이다.  
`isna` 의 경우 결측값이면 `True` 반환, 정상값이면 `False` 를 반환하며,  
`notna` 의 경우 결측값이면 `False` 반환, 정상값이면 `True` 를 반환한다.

__주의사항__: 무한값(`np.inf`) 나 띄워쓰기`' '`의 경우 결측값으로 판단하지 않는다.

#### isna

In [ ]:
df.head().isna()

#### notna

In [ ]:
df.head().notna()

#### 누락된 값의 수 구하기

In [ ]:
df.isna().sum()

#### 행 별로 누락된 값의 수 구하기

In [ ]:
df.isna().sum(axis=1)

__isnull() vs isna()__

동일한 함수이다.  
DB 에 저장된 결측치는 `null` 이라고 표시되는데, 이에 익숙한 사람들을 위해 `isna()` 의 별칭으로 `isnull()` 을 만들어 두었다.

---

이러한 결측치를 처리하는 2가지 방법이 존재한다.
- 1. 버리기(Drop) - dropna
- 2. 채우기(Fill) - fillna

## 5. dropna

첫 번째 방법으로, 누락된 값이 있는 행이나 열을 삭제하는 데 사용되는데, 다음 매개변수들이 존재한다.

참고사이트 - [pandas Docs - Dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html)

### axis
- __0__ or __'index'__ : 결측값이 있는 행을 삭제 -> default
- __1__ or __'columns'__ : 결측값이 있는 열을 삭제

In [ ]:
df = pd.read_csv('data/test_data_has_null.CSV', encoding='cp949')

In [ ]:
df.isna().sum()

`dropna()` 는 DataFrame 을 반환한다.  
원본 DataFrame 을 변경하고 싶으면 `inplace=True` 를 설정해준다

In [ ]:
df.dropna(axis=0)
df.isna().sum()

In [ ]:
df.dropna(axis=0, inplace=True)
df.isna().sum()

### how
- __any__: row 또는 column 에 NaN 값이 1개만 있어도 drop -> default
- __all__: row 또는 column 에 있는 모든 값이 NaN 이어야 drop

In [ ]:
df = pd.read_csv('data/test_data_has_null.CSV', encoding='cp949')
df.dropna(axis=0, how='all', inplace=True)
df.isna().sum()

### thresh
- 결측값이 아닌 열이 몇 개 이상일 경우 삭제하지 않는다는 임계치(기준)을 설정한다.
- `axis=0, thresh=3` 일 때, 결측값이 아닌 열이 3개 이상이면 삭제하지 않는다.

In [ ]:
df = pd.read_csv('data/test_data_has_null.CSV', encoding='cp949')
df

In [ ]:
df = pd.read_csv('data/test_data_has_null.CSV', encoding='cp949')
df.dropna(axis=0, thresh=4, inplace=True)
df.isna().sum()

In [ ]:
df.shape

## 6. fillna

결측값을 다른 적절한 값으로 채워준다.

In [ ]:
df.head()

In [ ]:
df['직업'].fillna('무직', inplace=True)
df.head()

In [ ]:
df.isna().sum()

## 7. groupby

열의 고유한 값을 기반으로 행을 그룹화한다.  
독립된 그룹에 대하여 __별도로 데이터를 처리__ 하거나 __그룹별 통계량__ 을 확인하고자 할 때 유용하다.  

groupby 는 아래 3단계로 이루어 진다

![groupby image](./data/pandas-groupby-split-apply-combine.svg)

이미지 출처: [www.w3resource.com](https://www.w3resource.com/)

__1. Split__: `.groupby()` 에서 정의한 컬럼 조건에 따라 독립된 그룹으로 나누는 단계. 위 사진에서는 `ID` 값을 기준으로 그룹으로 나누었고, 결과로 `X`, `Y`, `Z` 3개의 sub-group 으로 분할되었다.

__2. Apply__: sub-group 에 그룹별 함수를 적용하는 단계. 예시에서는 합계(sum) 함수를 적용하여 각 그룹 별 총계가 합산된 결과를 확인할 수 있다.

__3. Combine__: 각 sub-group 별로 함수가 적용된 결과를 종합하여 다시 하나의 테이블로 합침

이러한 과정을 이용하여 데이터 전처리/분석 시 유용하게 활용할 수 있다.

아래 예시는 `직업 별 평균 나이` 를 구하는 예시 코드이다.

In [ ]:
df = pd.read_csv('data/test_data_has_null.CSV', encoding='cp949')

In [ ]:
df.groupby('직업').mean()

__참고 - 경고문__: 위의 함수는 평균치를 구하는 함수이다. 평균치라는 것은 숫자에 대해서만(`numeric_only`) 작동이 가능하기 때문에, 숫자로 이루어진 열에만 계산을 적용하라는 뜻으로 `numeric_only` 옵션이 기본적으로 `True` 로 설정되어 있다.  
미래 버전에서는 해당 옵션 기본값이 `False` 로 변하므로, 직접 `True` 로 설정하여 버그를 없애라는 경고 문구이다.  
아래와 같이 옵션을 지정하면 버그가 없어진다.

In [ ]:
df.groupby('직업').mean(numeric_only=True)

위 코드는
1. 직업 별로 분할
2. 직업 별로 분할된 테이블의 숫자로 된 모든 열('나이') 의 평균 계산
3. 통합  
을 통해 위와 같은 결과가 나온다

하나의 예시만 더 해보자.

`직업 별 평균 나이` 에서 `성별을 추가적으로 구분하여 출력` 하는 예시 코드이다.

In [ ]:
df.groupby(['직업', '성별']).mean(numeric_only=True).reset_index()

이와 같이 여러 그룹으로 구분하여, 여러가지 통계적인 계산이나 수학 연산 등이 가능하다.  
아래 Docs 를 참고하여 여러 예제 코드를 사용해보자.

[Pandas Docs - Groupby](https://pandas.pydata.org/docs/reference/groupby.html) 참고

## 8. unique & nunique
- __unique__: 고유값의 종류를 모두 출력
- __nunique__: 고유값의 개수를 반환

In [ ]:
df['직업'].unique()

In [ ]:
df['직업'].nunique()

## 9. 정렬
- `sort_values()` 함수를 사용한다

9.1. 정렬의 기준은 기본적으로 오름차순이다

In [ ]:
df['직업'].sort_values()

9.2. 내림차순은 `ascending=False` 옵션을 추가해준다.

In [ ]:
df['직업'].sort_values(ascending=False)

9.3. 모든 컬럼을 보기 위해서는 `by` 옵션을 활용한다.

In [ ]:
df.sort_values(by=['나이'])

9.4. 두 가지 이상의 정렬을 하는 방법은 다음과 같다.  
- 직업 별 오름차순, 나이 별 내림차순
- ascending 옵션을 정렬 필드 순서대로 각각 지정해준다.

In [ ]:
df.sort_values(by=['직업', '나이'], ascending=[True, False])